In [1]:
from google.adk.agents import Agent
from google.genai import types
from prompt import (
    TRAVEL_ADVISOR_DESCRIPTION,
    TRAVEL_ADVISOR_INSTRUCTION,
)
from google.adk.tools.tool_context import ToolContext
from google.adk.sessions import DatabaseSessionService
from google.adk.runners import Runner

# Define a subclass to bypass Runner's module inspection for app_name alignment
class TravelAgent(Agent):
    pass

session_service = DatabaseSessionService(
    db_url="sqlite+aiosqlite:///./session.db"
)

session = await session_service.create_session(
    app_name="weather_agent",
    user_id="u_123",
    state={
        "user_name" : "Origogi"
    }
)


MODEL = "gemini-2.5-flash"


async def get_weather(tool_context: ToolContext, location: str):
    """Get current weather information for a location."""
    # Dummy implementation - returns mock data
    return {
        "location": location,
        "temperature": "22°C",
        "condition": "Partly cloudy",
        "humidity": "65%",
        "wind": "12 km/h",
        "forecast": "Mild weather with occasional clouds expected throughout the day",
    }


async def get_exchange_rate(
    tool_context: ToolContext, from_currency: str, to_currency: str, amount: float
):
    """Get exchange rate between two currencies.
    Args should always be from_currency str, to_currency str, amount flot
    """
    # Dummy implementation - returns mock data
    mock_rates = {
        ("USD", "EUR"): 0.92,
        ("USD", "GBP"): 0.79,
        ("USD", "JPY"): 149.50,
        ("USD", "KRW"): 1325.00,
        ("EUR", "USD"): 1.09,
        ("EUR", "GBP"): 0.86,
        ("GBP", "USD"): 1.27,
        ("JPY", "USD"): 0.0067,
        ("KRW", "USD"): 0.00075,
    }

    rate = mock_rates.get((from_currency, to_currency), 1.0)
    converted_amount = amount * rate

    return {
        "from_currency": from_currency,
        "to_currency": to_currency,
        "amount": amount,
        "exchange_rate": rate,
        "converted_amount": converted_amount,
        "timestamp": "2024-03-15 10:30:00 UTC",
    }


async def get_local_attractions(
    tool_context: ToolContext, location: str, category: str = "all"
):
    """Get popular attractions and points of interest for a location."""
    # Dummy implementation - returns mock data
    attractions = {
        "Paris": [
            {
                "name": "Eiffel Tower",
                "type": "landmark",
                "rating": 4.8,
                "description": "Iconic iron lattice tower",
            },
            {
                "name": "Louvre Museum",
                "type": "museum",
                "rating": 4.7,
                "description": "World's largest art museum",
            },
            {
                "name": "Arc de Triomphe",
                "type": "monument",
                "rating": 4.6,
                "description": "Historic triumphal arch",
            },
            {
                "name": "Notre-Dame",
                "type": "cathedral",
                "rating": 4.5,
                "description": "Medieval Catholic cathedral",
            },
            {
                "name": "Sacré-Cœur",
                "type": "basilica",
                "rating": 4.4,
                "description": "Romano-Byzantine basilica",
            },
        ],
        "Tokyo": [
            {
                "name": "Tokyo Tower",
                "type": "landmark",
                "rating": 4.5,
                "description": "Communications and observation tower",
            },
            {
                "name": "Senso-ji",
                "type": "temple",
                "rating": 4.6,
                "description": "Ancient Buddhist temple",
            },
            {
                "name": "Shibuya Crossing",
                "type": "landmark",
                "rating": 4.4,
                "description": "Busiest pedestrian crossing",
            },
            {
                "name": "Meiji Shrine",
                "type": "shrine",
                "rating": 4.7,
                "description": "Shinto shrine dedicated to Emperor Meiji",
            },
            {
                "name": "Tokyo Skytree",
                "type": "tower",
                "rating": 4.6,
                "description": "Broadcasting and observation tower",
            },
        ],
        "default": [
            {
                "name": "City Center",
                "type": "area",
                "rating": 4.2,
                "description": "Main downtown area",
            },
            {
                "name": "Historical Museum",
                "type": "museum",
                "rating": 4.3,
                "description": "Local history and culture",
            },
            {
                "name": "Central Park",
                "type": "park",
                "rating": 4.1,
                "description": "Main public park",
            },
            {
                "name": "Old Town",
                "type": "district",
                "rating": 4.4,
                "description": "Historic district with traditional architecture",
            },
            {
                "name": "Local Market",
                "type": "market",
                "rating": 4.0,
                "description": "Traditional local marketplace",
            },
        ],
    }

    location_attractions = attractions.get(location, attractions["default"])

    if category != "all":
        location_attractions = [
            a for a in location_attractions if a["type"] == category
        ]

    return {
        "location": location,
        "category": category,
        "attractions": location_attractions,
        "total_count": len(location_attractions)
    }


travel_advisor_agent = TravelAgent(
    name="TravelAdvisorAgent",
    description=TRAVEL_ADVISOR_DESCRIPTION,
    instruction=TRAVEL_ADVISOR_INSTRUCTION,
    tools=[
        get_weather,
        get_exchange_rate,
        get_local_attractions,
    ],
    model=MODEL,
    
)

runner = Runner(
    app_name="weather_agent",
    agent=travel_advisor_agent,
    session_service=session_service,
)

message = types.Content(
    role="user",
    parts=[
        types.Part(
            text="안녕 나는 서울에 놀러갈 계획이야, 두꺼운 옷을 챙겨갈까? 얇은 옷을 챙겨갈까?"
        )
    ]
)

async for event in runner.run_async(user_id="u_123", session_id=session.id, new_message=message):

    if event.is_final_response():
        print(event.content)
    else:
        print(event.get_function_calls())
        print(event.get_function_responses())


[FunctionCall(
  args={
    'location': 'Seoul'
  },
  id='adk-d2d0c969-f15b-4b31-9308-0876936b47b9',
  name='get_weather'
)]
[]
[]
[FunctionResponse(
  id='adk-d2d0c969-f15b-4b31-9308-0876936b47b9',
  name='get_weather',
  response=<dict len=6>
)]
parts=[Part(
  text="""Origogi님, 서울은 현재 22°C이며 부분적으로 흐리고 가끔 구름이 끼는 온화한 날씨가 예상됩니다. 바람은 시속 12km로 불고 습도는 65%입니다.

따라서 너무 두꺼운 옷보다는 가벼운 겉옷이나 얇은 옷 위주로 챙기시는 것이 좋겠습니다. 밤에는 기온이 조금 내려갈 수 있으니 가디건이나 얇은 재킷 하나 정도 챙기시면 유용할 거예요.

서울 여행에 대해 더 궁금한 점이 있으신가요? 예를 들어, 환전 정보나 가볼 만한 곳 등을 알려드릴 수 있습니다."""
)] role='model'
